In [ ]:
# Clone the Tensorflow Model Garden and installing the object detection packages
!git clone --depth 1 https://github.com/tensorflow/models

fatal: destination path 'models' already exists and is not an empty directory.


In [ ]:
# Installing the object detection packages
%%bash
sudo apt install -y protobuf-compiler
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Reading package lists...
Building dependency tree...
Reading state information...
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1684828 sha256=538abb33008e7e7496d13526a467a43e0e60c8a2bdcd43ccdde5af3b26ca541f
  Stored in directory: /tmp/pip-ephem-wheel-cache-dsv504le/wheels/fa/a4/d2/e9a5057e414fd46c8e543d2706cd836d64e1fcd9eccceb2329
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-py3-none-any.whl size=22257 sha256=0c392224c50235a06374e90fac9a162692dca43d30427a5810c8b44020a5a22f
  Stored in directory: /root/.cache/pip/wheels/d2/f1/1f/041add21dc9c4220157f1bd2bd6afe1f1a49524c3396b94401
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78544 sha256=7996d4e5d11c5fb5b83c8ba02c4e478f2b7c27c6c76b80b7f6a82f2767db7ce7
  Stored in directory: /root/.cache/pip/w



  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
multiprocess 0.70.12.2 requires dill>=0.3.4, but you have dill 0.3.1.1 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
# import library
from PIL import Image
from urllib.request import urlopen
import numpy as np
import tensorflow_hub as hub
from matplotlib import pyplot as plt
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import label_map_util
import cv2

In [ ]:
# Load object detection model from Tensorflow Hub
object_detector = hub.load("https://tfhub.dev/tensorflow/mask_rcnn/inception_resnet_v2_1024x1024/1")

# Map class label name with its id
label_path = './models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(label_path, use_display_name=True)

In [ ]:
# image_array has shape (H, W, 3)
def objectDetector(image_array):  
  # Convert the numppy array with shape (H, W, 3) to the shape (1, H, W, 3)
  img_numpy= np.expand_dims(image_array,0)  

  # Using loaded model to detect image
  detector_output = object_detector(img_numpy)

  # overlay bounding boxes and its label on the image
  viz_utils.visualize_boxes_and_labels_on_image_array(
        img_numpy[0],
        detector_output['detection_boxes'][0].numpy(),
        detector_output['detection_classes'][0].numpy().astype(int),
        detector_output['detection_scores'][0].numpy(),
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=100,
        min_score_thresh=.70,
        agnostic_mode=False,
        line_thickness=1)
  
  # Return the image with bounding boxes and class labels
  return img_numpy[0]

In [ ]:
# Mount Google Driver
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Working directory and data directory
working_dir='drive/MyDrive/Deeplearning_Colab/ComputerVision/'
data_dir=working_dir+'data/'

# Load video from its path
video = cv2.VideoCapture(data_dir+'walking_short.mp4')
#video = cv2.VideoCapture(0) #use this for webcam

# Define the resolutions of saved video
frame_width = int(video.get(3))
frame_height = int(video.get(4))   
size = (frame_width, frame_height)

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
out = cv2.VideoWriter(data_dir+'output.avi', fourcc, 10, size)

# Process each frame of the video
while True:
    read_success, frame = video.read()

    if(read_success):     
        # Detect object in each frame
        objectDetected_frame = objectDetector(frame)

        # write the frame with detected boxes and labels
        out.write(objectDetected_frame)
        # cv2.imshow(objectDetected_frame)

        # Wait for 1 ms
        key = cv2.waitKey(1)
        # Exit if press Escapse
        if key == 27:
            break

# Release resources and destroy all windows
video.release()
out.release()
cv2.destroyAllWindows()